In [1]:
import numpy as np 
import qiskit
from qiskit.quantum_info import purity

In [2]:
global qubit 
qubit = 4

In [3]:
# func to def labels
def labelCreation(qubit:int):
    labels = []
    for j in range(2**qubit):
        b = bin(j).replace("0b","")
        if len(b) < qubit:
            n = qubit - len(b)
            b = n*"0"+b
        labels.append(b)
    return labels

In [4]:
ρ_labels = labelCreation(qubit)
# ρ_pt_labels = labelCreation(qubit-1);

In [19]:
for j in range(len(ρ_labels)):
    print("j = ",j,"--> ", ρ_labels[j]);

j =  0 -->  000
j =  1 -->  001
j =  2 -->  010
j =  3 -->  011
j =  4 -->  100
j =  5 -->  101
j =  6 -->  110
j =  7 -->  111


In [25]:
coef = np.zeros(2**qubit)
coef[0] = 1/np.sqrt(2);
coef[5] = 1/np.sqrt(2);
# coef[10] = 1/2;
# coef[15] = -1/2;
for j in range(len(ρ_labels)):
    print("j = {}, coef = {}, label = {}".format(j,coef[j],ρ_labels[j]))

j = 0, coef = 0.7071067811865475, label = 000
j = 1, coef = 0.0, label = 001
j = 2, coef = 0.0, label = 010
j = 3, coef = 0.0, label = 011
j = 4, coef = 0.0, label = 100
j = 5, coef = 0.7071067811865475, label = 101
j = 6, coef = 0.0, label = 110
j = 7, coef = 0.0, label = 111


In [21]:
coef1 = np.zeros(2**qubit-1)
coef1[0] = 1/np.sqrt(2);
coef1[5] = 1/np.sqrt(2);
for j in range(len(ρ_pt_labels)):
    print("j = {}, coef = {}, label = {}".format(j,coef1[j],ρ_pt_labels[j]))

j = 0, coef = 0.7071067811865475, label = 00
j = 1, coef = 0.0, label = 01
j = 2, coef = 0.0, label = 10
j = 3, coef = 0.0, label = 11


In [11]:
coef2 = np.zeros(2**qubit-1)
coef2[2] = 1/np.sqrt(2)
coef2[7] = -1/np.sqrt(2)
for j in range(len(ρ_pt_labels)):
    print("j = {}, coef = {}, label = {}".format(j,coef2[j],ρ_pt_labels[j]))

j = 0, coef = 0.0, label = 000
j = 1, coef = 0.0, label = 001
j = 2, coef = 0.7071067811865475, label = 010
j = 3, coef = 0.0, label = 011
j = 4, coef = 0.0, label = 100
j = 5, coef = 0.0, label = 101
j = 6, coef = 0.0, label = 110
j = 7, coef = -0.7071067811865475, label = 111


In [22]:
# func to def state vector and dm 
def StateCreation(coef,labels,qubit):
    if np.isclose(np.linalg.norm(coef),1):
        ψ = qiskit.quantum_info.Statevector(np.zeros(2**qubit))
        for j in range(len(labels)):
            ψ += coef[j]*qiskit.quantum_info.Statevector.from_label(labels[j])
        ρ = qiskit.quantum_info.DensityMatrix(ψ)
    else:
        raise TypeError("The coefficient vector is not normalized")
    return ψ, ρ


In [26]:
ψ, ρ = StateCreation(coef, ρ_labels,qubit)
ψ.draw('latex')

<IPython.core.display.Latex object>

In [18]:
ψ1, ρ1 = StateCreation(coef1, ρ_pt_labels,qubit-1)
ψ1.draw('latex')

<IPython.core.display.Latex object>

In [20]:
ψ2, ρ2 = StateCreation(coef2, ρ_pt_labels, qubit-1)
ψ2.draw('latex')

<IPython.core.display.Latex object>

In [27]:
print(purity(ρ))
# print(purity(ρ1))
# print(purity(ρ2))

(0.9999999999999996+0j)


In [10]:
ρ1.data;

In [28]:
ρ.draw('latex')

<IPython.core.display.Latex object>

In [29]:
# ???    # [1,2,3]
def PartialTrace(ρ,L:list):
#     L.sort()
    pt0 = []
    pt1 = []
    pt2 = []
    pt3 = []
    if np.isin(0,L):
        pt0.append(ρ)
        
    if np.isin(1,L) and len(ρ.dims()) > 1:
        for j in range(len(ρ.dims())):
            pt1.append(qiskit.quantum_info.partial_trace(ρ,[j]))
    
    if np.isin(2,L) and len(ρ.dims()) > 2:
        for j in range(len(ρ.dims())-1):
            pt2.append(qiskit.quantum_info.partial_trace(ρ,[j,j+1]))
            if j+2 < len(ρ.dims()):
                pt2.append(qiskit.quantum_info.partial_trace(ρ,[j,j+2]))
                if j+3 < len(ρ.dims()):
                    pt2.append(qiskit.quantum_info.partial_trace(ρ,[j,j+3]))
                    
    if np.isin(3,L) and len(ρ.dims()) > 3:
        for j in range(len(ρ.dims())-2):
            pt3.append(qiskit.quantum_info.partial_trace(ρ,[j,j+1,j+2]))
            if j+3 < len(ρ.dims()):
                pt3.append(qiskit.quantum_info.partial_trace(ρ,[j,j+1,j+3]))
                if j+1 < len(ρ.dims())-2:
                    pt3.append(qiskit.quantum_info.partial_trace(ρ,[j,j+2,j+3]))
                    
    partial_trace = {
        "partial_trace_0" : pt0,
        "partial_trace_1" : pt1,
        "partial_trace_2" : pt2,
        "partial_trace_3" : pt3
    }
                        
    return partial_trace     

In [30]:
ρ_pt = PartialTrace(ρ, [1,2,3])
len(ρ_pt['partial_trace_1'])

3

In [34]:
ρ_pt.keys()

dict_keys(['partial_trace_0', 'partial_trace_1', 'partial_trace_2', 'partial_trace_3'])

In [65]:
print(ρ_pt["partial_trace_1"][0].is_valid())
print("purity= ", purity(ρ_pt["partial_trace_1"][0]))

True
purity=  (0.4999999999999998+0j)


In [39]:
ρ1.data.real

array([[0.5, 0. , 0. , 0. , 0. , 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0. , 0. , 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]])

In [42]:
0.5*ρ1 + 0.5*ρ2 == ρ_pt["partial_trace_1"][3]

True

In [58]:
a1 = ρ1.data.real
a2 = ρ2.data.real
A = np.transpose(np.array([a1.flatten(), a2.flatten()]))

In [62]:
B = np.reshape(ρ_pt["partial_trace_1"][3].data.real, [64,1])
B.shape

(64, 1)

In [64]:
from scipy import linalg

In [66]:
linalg.lstsq(A,B)

(array([[0.5],
        [0.5]]),
 array([2.56790659e-34]),
 2,
 array([1., 1.]))

In [67]:
# x = np.linalg.solve(A,B)
np.linalg.pinv(A) @ B

array([[0.5],
       [0.5]])

In [40]:
a0 = ρ1_pt["partial_trace_1"][0].data.real
print(np.linalg.matrix_rank(a0))

print(a0)

2
[[ 0.25  0.    0.    0.    0.    0.25  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.25  0.    0.    0.    0.   -0.25]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.25  0.    0.    0.    0.    0.25  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.   -0.25  0.    0.    0.    0.    0.25]]


In [39]:
a0 = ρ1_pt["partial_trace_1"][1].data.real
print(np.linalg.matrix_rank(a0))
print(a0)

2
[[ 0.25  0.    0.    0.25  0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.25  0.    0.    0.25  0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.25  0.    0.   -0.25]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.   -0.25  0.    0.    0.25]]


In [29]:
for j in range(len(ρ1_pt["partial_trace_1"])):
    print(purity(ρ1_pt["partial_trace_1"][j]).real)


0.5
0.5
0.5
0.5


In [15]:
# Function to compute EOF for a three-partite system
def EOF_3(ρ_pt): 
    eof_1 = []
    eof_2 = []
    entropy_1 = []
    entropy_2 = []
    if len(ρ_pt['partial_trace_1']) == 0:
        pass
    else:
        for j in range(len(ρ_pt['partial_trace_1'])):
            eof_1.append(qiskit.quantum_info.entanglement_of_formation(
                ρ_pt['partial_trace_1'][j]))
            entropy_1.append(qiskit.quantum_info.entropy(ρ_pt['partial_trace_1'][j]))
    
    if len(ρ_pt['partial_trace_2']) == 0:
        pass 
    else:
        for j in range(len(ρ_pt['partial_trace_2'])):
            eof_2.append(qiskit.quantum_info.entanglement_of_formation(
                ρ_pt['partial_trace_1'][j]))
            entropy_2.append(qiskit.quantum_info.entropy(ρ_pt['partial_trace_2'][j]))
            
            
    eof_1 = np.array(eof_1)
    eof_2 = np.array(eof_2)
    entropy_1 = np.array(entropy_1)
    entropy_2 = np.array(entropy_2)
    EOF = np.sum(eof_1+eof_2+entropy_1+entropy_2)/6
    return EOF

In [21]:
V = np.zeros((8,1))
D = []
start = 0;
stop = 1.1;
step = 0.2;
num = 0;

for v0 in np.arange(start, stop, step):
    V[0,0] = v0
    for v1 in np.arange(start, stop, step):
        V[1,0] = v1
        for v2 in np.arange(start, stop, step):
            V[2,0] = v2
            for v3 in np.arange(start, stop, step):
                V[3,0] = v3
                for v4 in np.arange(start, stop, step):
                    V[4,0] = v4
                    for v5 in np.arange(start, stop, step):
                        V[5,0] = v5
                        for v6 in np.arange(start, stop, step):
                            V[6,0] = v6
                            for v7 in np.arange(start, stop, step):
                                V[7,0] = v7
                                ρ = qiskit.quantum_info.DensityMatrix(np.outer(V,V))
                                num += 1
                                ρ_2 = qiskit.quantum_info.DensityMatrix(np.matmul(ρ.data,ρ.data))
                                if ρ.is_valid() and ρ_2.is_valid():
                                        D.append(ρ)
                                    
print(len(D))
print(num)

3536


In [ ]:
DD = [x.data for x in D]
np.save("Density_Matrix.npy", DD, allow_pickle=True)
# np.save("Density_Matrix.npy", np.array(DD, dtype=object), allow_pickle=True)
# D_read = np.load("Density_Matrix.npy", allow_pickle= True)

In [24]:
# D[1].purity()
# np.isclose(qiskit.quantum_info.purity(D[10]),1 )
# qiskit.quantum_info.DensityMatrix()

# Save in h5

In [66]:
import h5py

In [ ]:
hf = h5py.File('r3.h5','w')

In [159]:
f = h5py.File("data.h5", "w")
for j in range(len(e)):
    f.create_dataset(name="e_{}".format(j), data=e[j])
    
print(f.keys())
f.close()

<KeysViewHDF5 ['e_0', 'e_1', 'e_2']>


In [161]:
ff = h5py.File("data.h5", "r")
print(ff.keys())
W = []
# for j in range(len(ff.keys())):
#     W.append()
for j in range(len(ff.keys())):
    print(ff.keys())

<KeysViewHDF5 ['e_0', 'e_1', 'e_2']>


In [25]:
start = 0.0;
stop = 1.1;
step = 0.1;

S = []
for i in range(len(ρ1_pt['partial_trace_1'])):
    for j in range(len(D)):
        for k in range(len(D)):
            for p1 in np.arange(start, stop, step):
                if np.isclose(np.linalg.norm(ρ1_pt['partial_trace_1'][i] -\
                                             (p1*D[j] + (1-p1)*D[k]) ), 0.01):
                    S.append([i,ρ1_pt[i],D[j],D[k],p1,p2])

print(len(S))

KeyboardInterrupt: 

In [20]:
start = 0.0;
stop = 1.1;
step = 0.1;

S = []
for i in range(len(ρ1_pt['partial_trace_1'])):
    for j in range(len(D)):
        for k in range(len(D)):
            for p1 in np.arange(start, stop, step):
                if p1*D[j] + (1-p1)*D[k] == ρ1_pt['partial_trace_1'][i]:
                    S.append([i,ρ1_pt[i],D[j],D[k],p1,p2])

4

In [ ]:
start = 0.0;
stop = 1.1;
step = 0.1;

S0 = []
for j in range(len(D)):
    for k in range(len(D)):
        for p1 in np.arange(start, stop, step):
            if p1*D[j] + (1-p1)*D[k] == ρ1_pt['partial_trace_1'][0]:
                S0.append([i,ρ1_pt[i],D[j],D[k],p1,p2])

#  Convexhull

In [15]:
import numpy as np
from scipy.optimize import linprog
from scipy.spatial import ConvexHull 

In [17]:
M0

array([[0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]])

In [62]:
a = np.zeros((64,4))
a.shape

(64, 4)

In [94]:
a[:,0] = list(M0.flatten())
a[:,1] = list(M1.flatten())
a[:,2] = list(M2.flatten())
a[:,3] = list(M3.flatten())

In [95]:
a

array([[0.5 , 0.5 , 0.25, 0.25],
       [0.5 , 0.5 , 0.25, 0.25],
       [0.  , 0.  , 0.25, 0.25],
       [0.  , 0.  , 0.25, 0.25],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  ],
       [0.5 , 0.5 , 0.25, 0.25],
       [0.5 , 0.5 , 0.25, 0.25],
       [0.  , 0.  , 0.25, 0.25],
       [0.  , 0.  , 0.25, 0.25],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.25, 0.25],
       [0.  , 0.  , 0.25, 0.25],
       [0.  , 0.  , 0.25, 0.25],
       [0.  , 0.  , 0.25, 0.25],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.25, 0.25],
       [0.  , 0.  , 0.25, 0.25],
       [0.  , 0.  , 0.25, 0.25],
       [0.  , 0.  , 0.25, 0.25],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  ],
       [0.

(64, 4)

In [ ]:
qhull_options='QJ'

In [20]:
M0.shape

(8, 8)

In [27]:
hull = ConvexHull(M0[:,3:5], qhull_options='QJ')

In [28]:
hull.vertices

array([0, 6, 2, 4, 3, 5], dtype=int32)

In [29]:
hull.ndim

2

In [30]:
vertices = [M0[j] for j in hull.vertices]
vertices

[array([0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ]),
 array([0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0.])]

In [125]:
for a,b,c,d in vertices:
    print(a,b,c,d)

0.5 0.5 0.25 0.25
0.5 0.5 0.25 0.25
0.0 0.0 0.25 0.25
0.0 0.0 0.25 0.25
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.5 0.5 0.25 0.25
0.5 0.5 0.25 0.25
0.0 0.0 0.25 0.25
0.0 0.0 0.25 0.25
0.0 0.0 0.0 0.0
0.0 0.0 0.25 0.25
0.0 0.0 0.25 0.25
0.0 0.0 0.25 0.25
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.25 0.25
0.0 0.0 0.25 0.25
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
